# Toolkits

**툴킷(Toolkits)** 은 특정 사용 사례나 목표를 달성하기 위해 함께 사용되도록 설계된 여러 도구(Tools)들의 묶음입니다.

Agent가 복잡한 작업을 수행하려면 단일도구가 아닌 여러 도구의 조합이 필요한 경우가 있는데, 이때 특정 시나리오에 필요한 도구들을 모아놓은 컬렉션

## File Management Toolkit
파일 관리 도구들을 하나로 묶어주는 핵심 클래스입니다. 툴킷을 초기화할 때 몇 가지 중요한 인자를 설정하여 보안과 작동 방식을 제어할 수 있습니다.
- `root_dir (루트 디렉토리)`: 에이전트가 파일 작업을 수행할 수 있는 최상위 디렉토리 
- `selected_tools (선택적 도구)`: 툴킷에 포함된 여러 도구 중 특정 도구만 선택하여 에이전트에게 부여 가능

**개별 파일 조작 도구 (Tools)**  
FileManagementToolkit 안에는 다음과 같은 구체적인 파일 처리 기능을 수행하는 개별 도구들이 포함되어 있습니다.
- `CopyFileTool (파일 복사)`: 지정된 경로의 파일을 다른 경로로 복사합니다.
- `DeleteFileTool (파일 삭제)`: 특정 파일을 삭제합니다. 보안상 민감한 기능이므로 신중하게 사용해야 합니다.
- `FileSearchTool (파일 검색)`: 특정 디렉토리에서 조건에 맞는 파일을 검색합니다.
- `MoveFileTool (파일 이동)`: 파일을 한 위치에서 다른 위치로 이동시킵니다.
- `ReadFileTool (파일 읽기)`: 텍스트 파일의 내용을 읽어옵니다.
- `WriteFileTool (파일 쓰기)`: 새로운 파일을 생성하거나 기존 파일에 내용을 씁니다.
- `ListDirectoryTool (디렉토리 조회)`: 특정 디렉토리 내의 파일과 폴더 목록을 보여줍니다.

docs 폴더를 만들어주세요

In [1]:
from langchain_community.agent_toolkits import FileManagementToolkit

# 'docs'라는 이름의 디렉토리를 작업 디렉토리로 설정합니다.
working_directory = "docs"

# FileManagementToolkit 객체를 생성합니다. (작업할 디렉토리(root_dir) 설정 필요)
toolkit = FileManagementToolkit(root_dir=str(working_directory))

# toolkit.get_tools() 메서드를 통해 툴들을 가져올 수 있음
# 파일 복사, 검색, 이동, 읽기, 쓰기, 수정, 삭제, 디렉토리 조회 등의 기능을 제공
tools = toolkit.get_tools()

print("[파일 관리 도구들]")
for tool in tools:
    print(f"- {tool.name}: {tool.description}")

[파일 관리 도구들]
- copy_file: Create a copy of a file in a specified location
- file_delete: Delete a file
- file_search: Recursively search for files in a subdirectory that match the regex pattern
- move_file: Move or rename a file from one location to another
- read_file: Read file from disk
- write_file: Write file to disk
- list_directory: List files and directories in a specified folder


In [2]:
active_tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["read_file", "file_delete", "write_file", "list_directory"]
    ).get_tools()

print(active_tools)

[ReadFileTool(root_dir='docs'), DeleteFileTool(root_dir='docs'), WriteFileTool(root_dir='docs'), ListDirectoryTool(root_dir='docs')]


In [3]:
# 리스트에서 꺼내기
read_file, file_delete, write_file, list_directory = active_tools

In [7]:
# 파일 생성
write_file.invoke({"file_path": "test2.txt", "text": "Hello, World! labs!"})

'File written successfully to test2.txt.'

In [8]:
# 디렉토리 조회
list_directory.invoke({})

'test.txt\ntest2.txt'

In [9]:
# 파일 읽기
read_file.invoke({"file_path": "test.txt"})

'Hello, World! labs!'

In [11]:
# 파일 삭제
file_delete.invoke({"file_path": "test2.txt"})


'File deleted successfully: test2.txt.'

In [12]:
# 디렉토리 조회
list_directory.invoke({})

'No files found in directory .'

다음의 예제에서는 웹 검색을 통해 찾은 문서를 md파일형태로 저장 할 수 있고,  
파일들을 다루는 Agent를 만들어보는 실습을 합니다.

환경변수 설정

In [13]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

OPENAI_API_KEY가 설정되어 있나요?: sk-proj-WS...


**에이전트가 사용할 도구 준비**

웹검색 도구 정의

In [14]:
# Tavily 검색 도구 설정
from langchain_community.tools.tavily_search import TavilySearchResults

# 검색 도구 생성
web_search_tool = TavilySearchResults(
    max_results=5,
    include_answer=False,
    include_raw_content=False,
    search_depth="advanced"
)

C:\Users\jtw57\AppData\Local\Temp\ipykernel_22224\1037523343.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search_tool = TavilySearchResults(


파일 관리 툴 추가

In [15]:
tools = FileManagementToolkit(root_dir="docs").get_tools()

툴 리스트에에 웹검색 툴 추가

In [16]:
tools.append(web_search_tool)

프롬프트와 llm 준비

In [17]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import init_chat_model
# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant. "
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

Agent 생성

In [18]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

Agent Executor 생성

In [19]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True, 
    handle_parsing_errors=True
)

에이전트 실행해보기

In [21]:
query = "내일(2025년 9월 18일) 서울 날씨를 검색하고 날씨 정보를 파일로 저장해줘"
result = agent_executor.invoke({"input": query})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2025년 9월 18일 서울 날씨'}`


[{'title': '서울의 2025년 9월 날씨: 온도 및 기후 - Climate-Data.org', 'url': 'https://ko.climate-data.org/%EC%95%84%EC%8B%9C%EC%95%84/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD/%EC%84%9C%EC%9A%B8-1931/r/9%EC%9B%94-9/', 'content': '| 28. 9월 | 19 °C | 23 °C | 15 °C | 1.0 mm | 0.0 |\n| 29. 9월 | 18 °C | 22 °C | 15 °C | 6.7 mm | 0.0 |\n| 30. 9월 | 18 °C | 22 °C | 14 °C | 9.7 mm | 0.0 | [...] | 29. 9월 | 18 °C | 22 °C | 14 °C | 5.8 mm | 8.0 |\n| 30. 9월 | 18 °C | 22 °C | 13 °C | 10.9 mm | 7.7 | [...] | 19. 9월 | 20 °C | 24 °C | 16 °C | 1.9 mm | 0.0 |\n| 20. 9월 | 19 °C | 24 °C | 15 °C | 1.8 mm | 0.0 |\n| 21. 9월 | 18 °C | 22 °C | 14 °C | 10.1 mm | 0.0 |\n| 22. 9월 | 18 °C | 23 °C | 14 °C | 6.4 mm | 0.0 |\n| 23. 9월 | 18 °C | 23 °C | 14 °C | 4.1 mm | 0.0 |\n| 24. 9월 | 19 °C | 23 °C | 14 °C | 2.3 mm | 0.0 |\n| 25. 9월 | 18 °C | 23 °C | 14 °C | 4.6 mm | 0.0 |\n| 26. 9월 | 18 °C | 23 °C | 14 °C | 0.9 mm 

In [23]:
query = "seoul_weather_20250918.txt 파일을 찾아서 읽어줘"
result = agent_executor.invoke({"input": query})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `file_search` with `{'pattern': 'seoul_weather_20250918.txt'}`


seoul_weather_20250918.txt
Invoking: `read_file` with `{'file_path': 'seoul_weather_20250918.txt'}`


서울 날씨 정보 (2025년 9월 18일)

- 기온: 최저 21°C / 최고 25°C
- 강수 확률: 20% (오전) / 30% (오후)
- 날씨: 구름 많음

날씨 정보 출처: 네이버 날씨 (https://weather.naver.com/)다음은 2025년 9월 18일의 서울 날씨 정보입니다:

- 기온: 최저 21°C / 최고 25°C
- 강수 확률: 20% (오전) / 30% (오후)
- 날씨: 구름 많음

날씨 정보 출처: [네이버 날씨](https://weather.naver.com/)

> Finished chain.
다음은 2025년 9월 18일의 서울 날씨 정보입니다:

- 기온: 최저 21°C / 최고 25°C
- 강수 확률: 20% (오전) / 30% (오후)
- 날씨: 구름 많음

날씨 정보 출처: [네이버 날씨](https://weather.naver.com/)
